In [91]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


In [92]:

# Load data from JSON file
file_path = '../data/raw/2024_05_27-04_37_40-johnson-nv0_2024_03_12.txt'
with open(file_path, 'r') as file:
    data = json.load(file)

# Print the keys in the JSON data
print(data.keys())

# Accessing NV List and Keys
nv_list = data['nv_list']
print(f"Number of NVs: {len(nv_list)}")

# Accessing Coordinates of NVs
for nv in nv_list:
    print(f"Coordinates (Pixel): {nv['coords']['pixel']}")


dict_keys(['nv_list', 'num_steps', 'num_reps', 'num_runs', 'uwave_ind', 'uwave_freq', 'num_exps_per_rep', 'load_iq', 'step_ind_master_list', 'counts-units', 'counts', 'states', 'timestamp', 'config', 'opx_config'])
Number of NVs: 10
Coordinates (Pixel): [58.628, 139.616]
Coordinates (Pixel): [137.025, 74.662]
Coordinates (Pixel): [170.501, 132.597]
Coordinates (Pixel): [171.074, 49.877]
Coordinates (Pixel): [173.93, 78.505]
Coordinates (Pixel): [144.169, 163.787]
Coordinates (Pixel): [110.023, 87.942]
Coordinates (Pixel): [135.139, 104.013]
Coordinates (Pixel): [161.477, 105.335]
Coordinates (Pixel): [131.144, 129.272]


In [93]:

# Process counts and states
counts = np.array(data["counts"])
states = np.array(data["states"])
num_runs = counts.shape[2]

start, window = 1000, 500
counts = counts[:, :, start:start + window, :, :]
states = states[:, :, start:start + window, :, :]

exclude_inds = ()
num_nvs = len(nv_list)
nv_list = [nv_list[ind] for ind in range(num_nvs) if ind not in exclude_inds]
num_nvs = len(nv_list)
counts = np.delete(counts, exclude_inds, axis=1)

# Break down the counts array
sig_counts = np.array(counts[0])
ref_counts = np.array(counts[1])



In [94]:
# # Function to compute correlation matrix
# def compute_correlation_matrix(binary_counts):
#     return np.corrcoef(binary_counts)

# # Function to adjust threshold values to minimize correlation in reference data
# def adjust_thresholds(ref_counts, thresh_vals, lr=0.001, max_iter=1000, tol=1e-10):
#     for iter in range(max_iter):
#         bin_ref_counts = np.where(ref_counts > thresh_vals[:, None, None, None], 1, 0)
#         corr_matrix_ref = compute_correlation_matrix(bin_ref_counts.reshape(num_nvs, -1))
#         avg_corr = np.mean(np.abs(corr_matrix_ref[np.triu_indices(num_nvs, k=1)]))

#         if avg_corr < tol:
#             print(f"Convergence reached at iteration {iter} with average correlation {avg_corr}")
#             break

#         gradient = np.sum(corr_matrix_ref, axis=1) - np.diag(corr_matrix_ref)
#         thresh_vals -= lr * gradient

#     return thresh_vals
# Function to compute correlation matrix
def compute_correlation_matrix(binary_counts):
    return np.corrcoef(binary_counts)

# Improved function to adjust threshold values to minimize correlation in reference data
def adjust_thresholds(ref_counts, thresh_vals, lr=0.001, beta=0.3, max_iter=1000, tol=1e-10):
    velocity = np.zeros_like(thresh_vals)
    for iter in range(max_iter):
        bin_ref_counts = np.where(ref_counts > thresh_vals[:, None, None, None], 1, 0)
        corr_matrix_ref = compute_correlation_matrix(bin_ref_counts.reshape(num_nvs, -1))
        avg_corr = np.mean(np.abs(corr_matrix_ref[np.triu_indices(num_nvs, k=1)]))

        if avg_corr < tol:
            print(f"Convergence reached at iteration {iter} with average correlation {avg_corr}")
            break

        gradient = np.sum(corr_matrix_ref, axis=1) - np.diag(corr_matrix_ref)
        velocity = beta * velocity + (1 - beta) * gradient
        thresh_vals -= lr * velocity

        # Ensure thresholds stay within a valid range
        thresh_vals = np.clip(thresh_vals, np.min(ref_counts), np.max(ref_counts))

    return thresh_vals
# Placeholder function for threshold_counts (needs proper definition based on your context)
def threshold_counts(nv_list, sig_counts, ref_counts, dynamic_thresh=True):
    if dynamic_thresh:
        initial_thresholds = np.array([nv['threshold'] for nv in data['nv_list']])
        thresholds = adjust_thresholds(ref_counts, initial_thresholds)
        sig_counts = np.where(sig_counts > thresholds[:, None, None, None], 1, 0)
        ref_counts = np.where(ref_counts > thresholds[:, None, None, None], 1, 0)
    return sig_counts, ref_counts

# Adjust thresholds and calculate correlations
sig_counts, ref_counts = threshold_counts(nv_list, sig_counts, ref_counts, dynamic_thresh=True)

flattened_sig_counts = [sig_counts[ind].flatten() for ind in range(num_nvs)]
flattened_ref_counts = [ref_counts[ind].flatten() for ind in range(num_nvs)]

In [ ]:
sig_corr_coeffs = np.corrcoef(flattened_sig_counts)
ref_corr_coeffs = np.corrcoef(flattened_ref_counts)

diff_corr_coeffs = np.cov(flattened_sig_counts) - np.cov(flattened_ref_counts)

spin_flips = np.array([-1 if nv.get('spin_flip', False) else +1 for nv in nv_list])
ideal_sig_corr_coeffs = np.outer(spin_flips, spin_flips).astype(float)

# Replace diagonals with nan
vals = [sig_corr_coeffs, diff_corr_coeffs, ref_corr_coeffs, ideal_sig_corr_coeffs]
for val in vals:
    np.fill_diagonal(val, np.nan)

print(np.nanmean(ref_corr_coeffs) / np.nanmean(np.abs(sig_corr_coeffs)))



In [ ]:
import os
# Define the output directory
output_dir = '../data/correlation_matrix'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save correlation matrices
np.save(os.path.join(output_dir, 'diff_corr_coeffs.npy'), diff_corr_coeffs)
# np.save(os.path.join(output_dir, 'correlation_matrix_signal.npy'), correlation_matrix_signal)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from matplotlib.ticker import MaxNLocator

def plot_correlation_matrices(vals, titles, cbar_maxes):
    figs = []
    for ind in range(len(vals)):
        fig, ax = plt.subplots()
        cax = ax.matshow(
            vals[ind],
            cmap="RdBu_r",
            norm=TwoSlopeNorm(vmin=-cbar_maxes[ind], vcenter=0, vmax=cbar_maxes[ind]),
        )
        fig.colorbar(cax)
        ax.set_title(titles[ind])
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))

        # Replace NaNs with gray color
        data = np.ma.masked_invalid(vals[ind])
        cmap = plt.cm.RdBu_r
        cmap.set_bad(color='gray')
        cax = ax.matshow(data, cmap=cmap, norm=TwoSlopeNorm(vmin=-cbar_maxes[ind], vcenter=0, vmax=cbar_maxes[ind]))

        figs.append(fig)
    
    return figs

# Example usage:
vals = [sig_corr_coeffs, diff_corr_coeffs, ref_corr_coeffs, ideal_sig_corr_coeffs]
titles = ["Signal", "Difference", "Reference", "Ideal signal"]
cbar_maxes = [np.nanmax(np.abs(sig_corr_coeffs)), np.nanmax(np.abs(diff_corr_coeffs)), np.nanmax(np.abs(ref_corr_coeffs)), 1]

figures = plot_correlation_matrices(vals, titles, cbar_maxes)

# Display all generated plots
for fig in figures:
    fig.tight_layout()
    plt.show()

plt.show()

In [ ]:

# Load data from JSON file
file_path = '../data/raw/2024_05_27-04_37_40-johnson-nv0_2024_03_12.txt'
with open(file_path, 'r') as file:
    data = json.load(file)
# Print the keys in the JSON data
print(data.keys())

In [ ]:
# Example: Accessing NV List and Keys
nv_list = data['nv_list']
print(f"Number of NVs: {len(nv_list)}")

# Example: Accessing Coordinates of NVs
for nv in nv_list:
    # print(f"NV Name: {nv['name']}")
    print(f"Coordinates (Pixel): {nv['coords']['pixel']}")
    # print(f"Coordinates (Global): {nv['coords']['global']}")

In [ ]:
# Function to compute binary counts with thresholding
def apply_thresholding(counts, thresholds):
    return np.where(counts > thresholds[:, None], 1, 0)

# Iteratively adjust threshold values to minimize correlation in reference data
def adjust_thresholds(reference_counts, threshold_values, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
    for iteration in range(max_iterations):
        binary_reference_counts = apply_thresholding(reference_counts, threshold_values)
        correlation_matrix_reference = compute_correlation_matrix(binary_reference_counts)
        avg_correlation = np.mean(np.abs(correlation_matrix_reference[np.triu_indices(num_nvs, k=1)]))

        if avg_correlation < tolerance:
            print(f"Convergence reached at iteration {iteration} with average correlation {avg_correlation}")
            break

        # Adjust thresholds based on correlation matrix
        gradient = np.sum(correlation_matrix_reference, axis=1) - np.diag(correlation_matrix_reference)
        threshold_values -= learning_rate * gradient

    return threshold_values

sig_counts, ref_counts = threshold_counts(
    nv_list, sig_counts, ref_counts, None, dynamic_thresh=True
)

# Calculate the correlations
flattened_sig_counts = [sig_counts[ind].flatten() for ind in range(num_nvs)]
flattened_ref_counts = [ref_counts[ind].flatten() for ind in range(num_nvs)]


num_shots = len(flattened_ref_counts[0])

In [ ]:

    sig_corr_coeffs = np.nan_corr_coef(flattened_sig_counts)
    ref_corr_coeffs = np.nan_corr_coef(flattened_ref_counts)


    diff_corr_coeffs = np.cov(flattened_sig_counts) - np.cov(flattened_ref_counts)

    spin_flips = np.array([-1 if nv.spin_flip else +1 for nv in nv_list])
    # spin_flips = np.array(
    #     [-1 if ind in [0, 1, 4, 6] else +1 for ind in range(num_nvs)]
    # )  # MCC
    ideal_sig_corr_coeffs = np.outer(spin_flips, spin_flips)
    ideal_sig_corr_coeffs = ideal_sig_corr_coeffs.astype(float)

    # Replace diagonals (Cii=1) with nan so they don't show
    vals = [sig_corr_coeffs, diff_corr_coeffs, ref_corr_coeffs, ideal_sig_corr_coeffs]
    for val in vals:
        np.fill_diagonal(val, np.nan)

    print(np.nanmean(ref_corr_coeffs) / np.nanmean(np.abs(sig_corr_coeffs)))



    # Make the colorbar symmetric about 0
    sig_max = np.nanmax(np.abs(sig_corr_coeffs))
    ref_max = np.nanmax(np.abs(ref_corr_coeffs))
    diff_max = np.nanmax(np.abs(diff_corr_coeffs))


In [ ]:

    figs = []
    titles = ["Signal", "Difference", "Reference", "Ideal signal"]
    cbar_maxes = [sig_max, diff_max, sig_max, 1]
    for ind in range(len(vals)):
        coors = vals[ind]  # Replace diagonals (Cii=1) with nan so they don't show
        np.fill_diagonal(coors, np.nan)
        fig, ax = plt.subplots()
        cbar_max = cbar_maxes[ind]
        # cbar_max = 0.032
        kpl.imshow(
            ax,
            vals[ind],
            title=titles[ind],
            cbar_label="Covariance" if ind == 1 else "Correlation coefficient",
            cmap="RdBu_r",
            vmin=-cbar_max,
            vmax=cbar_max,
            nan_color=kpl.KplColors.GRAY,
        )
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))
        figs.append(fig)

In [ ]:
# Number of NVs
num_nvs = len(data['nv_list'])
# Extract reference and signal counts
reference_counts = data['counts'][0]
signal_counts = data['counts'][1]

#Make numpy array reference and signal counts
reference_counts = np.array(data['counts'][0])
signal_counts = np.array(data['counts'][1])

# Verify the shape of counts arrays
print(f"Shape of reference_counts: {reference_counts.shape}")
print(f"Shape of signal_counts: {signal_counts.shape}")
# Flatten or reshape the counts data if needed (removing singleton dimension)
reference_counts = np.squeeze(reference_counts)  # This removes singleton dimensions
signal_counts = np.squeeze(signal_counts)
# Verify the shape of counts arrays
print(f"Shape of reference_counts: {reference_counts.shape}")
print(f"Shape of signal_counts: {signal_counts.shape}")

In [ ]:
# Average over the repetitions (3rd dimension) to get a (10, 2000) array for each count
avg_reference_counts = np.mean(reference_counts, axis=2)
avg_signal_counts = np.mean(signal_counts, axis=2)
# print(avg_signal_counts)
# Threshold values
threshold_values = np.array([nv['threshold'] for nv in data['nv_list']])
print(threshold_values)
# Apply thresholding to reference and signal counts
binary_reference_counts = np.where(avg_reference_counts > threshold_values[:, None], 1, 0)
binary_signal_counts = np.where(avg_signal_counts > threshold_values[:, None], 1, 0)
# print(binary_signal_counts)

In [ ]:
# Function to compute binary counts with thresholding
def apply_thresholding(counts, thresholds):
    return np.where(counts > thresholds[:, None], 1, 0)

# Iteratively adjust threshold values to minimize correlation in reference data
def adjust_thresholds(reference_counts, threshold_values, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
    for iteration in range(max_iterations):
        binary_reference_counts = apply_thresholding(reference_counts, threshold_values)
        correlation_matrix_reference = compute_correlation_matrix(binary_reference_counts)
        avg_correlation = np.mean(np.abs(correlation_matrix_reference[np.triu_indices(num_nvs, k=1)]))

        if avg_correlation < tolerance:
            print(f"Convergence reached at iteration {iteration} with average correlation {avg_correlation}")
            break

        # Adjust thresholds based on correlation matrix
        gradient = np.sum(correlation_matrix_reference, axis=1) - np.diag(correlation_matrix_reference)
        threshold_values -= learning_rate * gradient

    return threshold_values

# Adjust thresholds
adjusted_threshold_values = adjust_thresholds(avg_reference_counts, threshold_values)
print(f"Adjusted threshold values: {adjusted_threshold_values}")


In [ ]:

# Apply thresholding to reference and signal counts using adjusted thresholds
binary_reference_counts = apply_thresholding(avg_reference_counts, adjusted_threshold_values)
binary_signal_counts = apply_thresholding(avg_signal_counts, adjusted_threshold_values)


# Compute correlation matrices for shot-to-shot correlations
correlation_matrix_reference = compute_correlation_matrix(binary_reference_counts)
correlation_matrix_signal = compute_correlation_matrix(binary_signal_counts)

# Set the range for the colormap
vmin = -0.1
vmax = 0.2

# Plot correlation matrix for reference counts
# plt.imshow(correlation_matrix_reference, cmap='bwr', interpolation='nearest', vmin=vmin, vmax=vmax)
# Replace diagonal elements with gray color
np.fill_diagonal(correlation_matrix_reference, np.nan)  # Replace diagonal elements with NaN
plt.imshow(correlation_matrix_reference, cmap='bwr', interpolation='nearest')
plt.colorbar()
plt.title('Correlation Matrix (Reference Counts)')
plt.show()

# Plot correlation matrix for signal counts
np.fill_diagonal(correlation_matrix_signal,np.nan)
plt.imshow(correlation_matrix_signal, cmap='bwr', interpolation='nearest', vmin=vmin, vmax=vmax)
plt.colorbar()
plt.title('Correlation Matrix (Signal Counts)')
plt.show()

In [ ]:
# Save processed data
output_dir = '../data/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path_reference = os.path.join(output_dir, 'processed_binary_reference_counts.npy')
output_path_signal = os.path.join(output_dir, 'processed_binary_signal_counts.npy')

np.save(output_path_reference, binary_reference_counts)
np.save(output_path_signal, binary_signal_counts)

print(f"Processed binary reference counts saved to: {output_path_reference}")
print(f"Processed binary signal counts saved to: {output_path_signal}")

# Save the correlation matrices
correlation_matrix_path_reference = os.path.join(output_dir, 'correlation_matrix_reference.npy')
correlation_matrix_path_signal = os.path.join(output_dir, 'correlation_matrix_signal.npy')

np.save(correlation_matrix_path_reference, correlation_matrix_reference)
np.save(correlation_matrix_path_signal, correlation_matrix_signal)

print(f"Correlation matrix for reference counts saved to: {correlation_matrix_path_reference}")
print(f"Correlation matrix for signal counts saved to: {correlation_matrix_path_signal}")

In [ ]:
# Compute correlation matrices for shot-to-shot correlations
correlation_matrix_reference = compute_correlation_matrix(binary_reference_counts)
correlation_matrix_signal = compute_correlation_matrix(binary_signal_counts)

# Plot correlation matrices
plt.imshow(correlation_matrix_reference, cmap='bwr', interpolation='nearest')
plt.colorbar()
plt.title('Correlation Matrix (Reference Counts)')
plt.show()

plt.imshow(correlation_matrix_signal, cmap='bwr', interpolation='nearest')
plt.colorbar()
plt.title('Correlation Matrix (Signal Counts)')
plt.show()


In [ ]:
# Define the output directory
output_dir = '../data/correlation_matrix'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save correlation matrices
np.save(os.path.join(output_dir, 'correlation_matrix_reference.npy'), correlation_matrix_reference)
np.save(os.path.join(output_dir, 'correlation_matrix_signal.npy'), correlation_matrix_signal)

In [ ]:
# Compute correlation matrices
correlation_matrix_reference = np.corrcoef(averaged_reference_counts)
correlation_matrix_signal = np.corrcoef(averaged_signal_counts)

# Plot resized correlation matrices
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

cax0 = axs[0].imshow(correlation_matrix_reference, cmap='hot', interpolation='nearest')
axs[0].set_title('Reference Counts Correlation Matrix')
axs[0].set_xlabel('NVs')
axs[0].set_ylabel('NVs')
fig.colorbar(cax0, ax=axs[0])

cax1 = axs[1].imshow(correlation_matrix_signal, cmap='hot', interpolation='nearest')
axs[1].set_title('Signal Counts Correlation Matrix')
axs[1].set_xlabel('NVs')
axs[1].set_ylabel('NVs')
fig.colorbar(cax1, ax=axs[1])

plt.tight_layout()
plt.show()

In [ ]:
# Threshold values
threshold_values = np.array([nv['threshold'] for nv in data['nv_list']])
# Apply thresholding to reference and signal counts
binary_reference_counts = np.where(averaged_reference_counts > threshold_values[:, None], 1, 0)
binary_signal_counts = np.where(averaged_signal_counts > threshold_values[:, None], 1, 0)
print(binary_reference_counts)
print(binary_reference_counts)

In [ ]:
# Compute correlation matrices
correlation_matrix_reference = np.corrcoef(binary_reference_counts)
correlation_matrix_signal = np.corrcoef(binary_signal_counts)

# Plot resized correlation matrices
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

cax0 = axs[0].imshow(correlation_matrix_reference, cmap='hot', interpolation='nearest')
axs[0].set_title('Reference Counts Correlation Matrix')
axs[0].set_xlabel('NVs')
axs[0].set_ylabel('NVs')
fig.colorbar(cax0, ax=axs[0])

cax1 = axs[1].imshow(correlation_matrix_signal, cmap='hot', interpolation='nearest')
axs[1].set_title('Signal Counts Correlation Matrix')
axs[1].set_xlabel('NVs')
axs[1].set_ylabel('NVs')
fig.colorbar(cax1, ax=axs[1])

plt.tight_layout()
plt.show()

In [ ]:
# Define the output directory
output_dir = '../data/correlation_matrix'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save correlation matrices
np.save(os.path.join(output_dir, 'correlation_matrix_reference.npy'), correlation_matrix_reference)
np.save(os.path.join(output_dir, 'correlation_matrix_signal.npy'), correlation_matrix_signal)